# Eat Safe, Love

## Notebook Set Up

In [71]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import json 
import requests

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_foods']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusinessName': 1, 'scores.Hygiene': 1}
hygiene_scores = list(establishments.find(query,fields))

# Use count_documents to display the number of documents in the result
print("Number of establishments with a hygiene score of 20 are:", establishments.count_documents(query))

# Display the first document in the results using pprint
for hscores in range(1):
    pprint(hygiene_scores[hscores])

Number of establishments with a hygiene score of 20 are: 41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('63d1ef270eddd0949a7badff'),
 'scores': {'Hygiene': 20}}


In [87]:
# Convert the result to a Pandas DataFrame
cleanly = pd.DataFrame(hygiene_scores)
# Display the number of rows in the DataFrame
pprint(cleanly.count())
# Display the first 10 rows of the DataFrame
cleanly.head(10)

_id             41
BusinessName    41
scores          41
dtype: int64


,_id,BusinessName,scores
0,63d1ef270eddd0949a7badff,The Chase Rest Home,{'Hygiene': 20}
1,63d1ef270eddd0949a7bb181,Brenalwood,{'Hygiene': 20}
2,63d1ef270eddd0949a7bb48a,Melrose Hotel,{'Hygiene': 20}
3,63d1ef270eddd0949a7bb673,Seaford Pizza,{'Hygiene': 20}
4,63d1ef270eddd0949a7bb684,Golden Palace,{'Hygiene': 20}
5,63d1ef270eddd0949a7bc031,Ashby's Butchers,{'Hygiene': 20}
6,63d1ef270eddd0949a7bc249,South Sea Express Cuisine,{'Hygiene': 20}
7,63d1ef280eddd0949a7bd775,Golden Palace,{'Hygiene': 20}
8,63d1ef280eddd0949a7bdbb1,The Tulip Tree,{'Hygiene': 20}
9,63d1ef280eddd0949a7be3c5,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [273]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': 'London'}, 
         'RatingValue':{'$gte': '4'}}
fields = {'BusinessName': 1, 'LocalAuthorityName': 1, 'RatingValue': 1} 
est_found = list(establishments.find(query,fields))

# Use count_documents to display the number of documents in the result
print("The number of documents with a Rating Value 4 and 5 in London are:", establishments.count_documents(query))
# Display the first document in the results using pprint

for estvalue in range(1):
    pprint(est_found[estvalue])


The number of documents with a Rating Value 4 and 5 in London are: 34
{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': '4',
 '_id': ObjectId('63d1ef270eddd0949a7bc819')}


In [274]:
# Convert the result to a Pandas DataFrame
ratings_london_df = pd.DataFrame(est_found)
# Display the number of rows in the DataFrame
pprint(ratings_london_df.count())
# Display the first 10 rows of the DataFrame
ratings_london_df.head(10)

_id                   34
BusinessName          34
RatingValue           34
LocalAuthorityName    34
dtype: int64


,_id,BusinessName,RatingValue,LocalAuthorityName
0,63d1ef270eddd0949a7bc819,Charlie's,4,City of London Corporation
1,63d1ef270eddd0949a7bcb3f,Mv City Cruises Erasmus,5,City of London Corporation
2,63d1ef280eddd0949a7bd68b,Benfleet Motor Yacht Club,4,City of London Corporation
3,63d1ef280eddd0949a7be489,Tilbury Seafarers Centre,5,City of London Corporation
4,63d1ef280eddd0949a7be48e,Coombs Catering t/a The Lock and Key,5,City of London Corporation
5,63d1ef280eddd0949a7bebb7,Mv Valulla,5,City of London Corporation
6,63d1ef280eddd0949a7c0c9d,Froottree,AwaitingInspection,City of London Corporation
7,63d1ef280eddd0949a7c0ca4,Tereza Joanne,5,City of London Corporation
8,63d1ef280eddd0949a7c106b,Brick Lane Brews,4,City of London Corporation
9,63d1ef280eddd0949a7c1084,The Nuance Group (UK) Limited,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [275]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = pd.DataFrame(db.establishments.find_one({'BusinessName': 'Penang Flavours'},{'geocode.latitude'}))
longitude = pd.DataFrame(db.establishments.find_one({'BusinessName': 'Penang Flavours'},{'geocode.longitude'}))
# Creating a float 
lat_geo = ((latitude['geocode']) + (degree_search))[0]
lat_geo1 = ((latitude['geocode']) - (degree_search))[0]
long_geo = ((longitude['geocode']) + (degree_search))[0]
long_geo1 = ((longitude['geocode']) - (degree_search))[0]


In [276]:
#Checking if calculations from variables 
print(lat_geo)
print(lat_geo1)
print(long_geo)
print(long_geo1)

51.500142
51.480142
0.09383999999999999
0.07384


In [291]:
# The query 
query1 = {'geocode.latitude': {'$gt' : lat_geo1, '$lt': lat_geo}, 'geocode.longitude' : {'$gt' : long_geo1, '$lt': long_geo}, 'RatingValue': {'$regex': '5'}}
fields = {'BusinessName': 1, 'RatingValue': 1, 'scores.Hygiene': 1, 'geocode': 1}
sort = [("scores.Hygiene", 1)]

# Print the results

results = list(establishments.find(query1, fields).sort(sort))

for result in range(5):
    pprint(results[result])



{'BusinessName': 'Iceland',
 'RatingValue': '5',
 '_id': ObjectId('63d1ef280eddd0949a7c0630'),
 'geocode': {'latitude': 51.4871482849121, 'longitude': 0.0924199968576431},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Howe and Co Fish and Chips - Van 17',
 'RatingValue': '5',
 '_id': ObjectId('63d1ef280eddd0949a7c063e'),
 'geocode': {'latitude': 51.4875335693359, 'longitude': 0.0925370007753372},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Volunteer',
 'RatingValue': '5',
 '_id': ObjectId('63d1ef280eddd0949a7c065c'),
 'geocode': {'latitude': 51.4873437, 'longitude': 0.09208},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Plumstead Manor Nursery',
 'RatingValue': '5',
 '_id': ObjectId('63d1ef280eddd0949a7c0676'),
 'geocode': {'latitude': 51.481517791748, 'longitude': 0.0859939977526665},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Lumbini Grocery Ltd T/A Al-Iman',
 'RatingValue': '5',
 '_id': ObjectId('63d1ef280eddd0949a7c0678'),
 'geocode': {'latitude': 51.4871632, 'longitude': 0.09162

In [289]:
# Convert result to Pandas DataFrame
top_5 = pd.DataFrame(results)

top_5.head(5)

,_id,BusinessName,RatingValue,scores,geocode
0,63d1ef280eddd0949a7c0630,Iceland,5,{'Hygiene': 0},{'latitude': 51.4871482849121}
1,63d1ef280eddd0949a7c063e,Howe and Co Fish and Chips - Van 17,5,{'Hygiene': 0},{'latitude': 51.4875335693359}
2,63d1ef280eddd0949a7c065c,Volunteer,5,{'Hygiene': 0},{'latitude': 51.4873437}
3,63d1ef280eddd0949a7c0676,Plumstead Manor Nursery,5,{'Hygiene': 0},{'latitude': 51.481517791748}
4,63d1ef280eddd0949a7c0678,Lumbini Grocery Ltd T/A Al-Iman,5,{'Hygiene': 0},{'latitude': 51.4871632}


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [301]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
hygiene_query = {'$match': {'scores.Hygiene': 0}}
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id':{'LocalAuthorityName': '$LocalAuthorityName'},
                          'count': {'$sum': 1}
                          }
               }
# 3. Sorts the matches from highest to lowest
sort_query = {'$sort': {'count': +1}}
# Print the number of documents in the result
pipeline = [hygiene_query, group_query, sort_query]
# Print the first 10 results
results = list(establishments.aggregate(pipeline))
pprint(results)

[{'_id': {'LocalAuthorityName': 'Broxbourne'}, 'count': 1},
 {'_id': {'LocalAuthorityName': 'Kensington and Chelsea'}, 'count': 1},
 {'_id': {'LocalAuthorityName': 'Dorset'}, 'count': 1},
 {'_id': {'LocalAuthorityName': 'Sunderland'}, 'count': 1},
 {'_id': {'LocalAuthorityName': 'North Norfolk'}, 'count': 1},
 {'_id': {'LocalAuthorityName': 'Reading'}, 'count': 1},
 {'_id': {'LocalAuthorityName': 'Mid Sussex'}, 'count': 5},
 {'_id': {'LocalAuthorityName': 'North Hertfordshire'}, 'count': 6},
 {'_id': {'LocalAuthorityName': 'Tower Hamlets'}, 'count': 9},
 {'_id': {'LocalAuthorityName': 'Ipswich'}, 'count': 21},
 {'_id': {'LocalAuthorityName': 'City of London Corporation'}, 'count': 28},
 {'_id': {'LocalAuthorityName': 'Tandridge'}, 'count': 42},
 {'_id': {'LocalAuthorityName': 'Lewisham'}, 'count': 65},
 {'_id': {'LocalAuthorityName': 'Rother'}, 'count': 80},
 {'_id': {'LocalAuthorityName': 'Wealden'}, 'count': 93},
 {'_id': {'LocalAuthorityName': 'Babergh'}, 'count': 97},
 {'_id': {'Lo

In [307]:
# Convert the result to a Pandas DataFrame
localauthority_df = pd.DataFrame(results)
normalized_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print("The number of rows in DataFrame: ", len(normalized_df))
# Display the first 10 rows of the DataFrame
normalized_df.head(10)

The number of rows in DataFrame:  55


,count,_id.LocalAuthorityName
0,1,Broxbourne
1,1,Kensington and Chelsea
2,1,Dorset
3,1,Sunderland
4,1,North Norfolk
5,1,Reading
6,5,Mid Sussex
7,6,North Hertfordshire
8,9,Tower Hamlets
9,21,Ipswich
